In [ ]:
%pip install -q langchain langchain-google-vertexai google-cloud-aiplatform google-auth pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:
from google.colab import files
import os
import json

In [ ]:
print("Please upload your GCP service-account JSON key file (recommended: a project-level service account with Vertex AI permissions).")
uploaded = files.upload()

Please upload your GCP service-account JSON key file (recommended: a project-level service account with Vertex AI permissions).


Saving food-donation-agent-76f6c6f468d6.json to food-donation-agent-76f6c6f468d6 (1).json


In [ ]:
if len(uploaded) == 0:
    raise SystemExit("No file uploaded. Upload service account JSON and re-run this cell.")

In [ ]:
sa_file = list(uploaded.keys())[0]
SA_KEY_PATH = f"/content/{sa_file}"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SA_KEY_PATH
print("Set GOOGLE_APPLICATION_CREDENTIALS =", SA_KEY_PATH)

Set GOOGLE_APPLICATION_CREDENTIALS = /content/food-donation-agent-76f6c6f468d6 (1).json


In [ ]:
PROJECT_ID = input("Enter your GCP project id (e.g. my-project-123): ").strip()
REGION = input("Enter Vertex AI region (e.g. us-central1) [default us-central1]: ").strip() or "us-central1"

Enter your GCP project id (e.g. my-project-123): food-donation-agent
Enter Vertex AI region (e.g. us-central1) [default us-central1]: us-central1


In [ ]:
os.environ["GCLOUD_PROJECT"] = PROJECT_ID
os.environ["GCP_PROJECT"] = PROJECT_ID

In [ ]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)
print("Vertex AI initialized for project:", PROJECT_ID, "region:", REGION)

Vertex AI initialized for project: food-donation-agent region: us-central1


In [ ]:
# ===============================
# 4️⃣ Import LangChain + VertexAI LLM
# ===============================
try:
    from langchain_google_vertexai import VertexAI
except Exception:
    from langchain_google_vertexai.llms import VertexAI

model_name = input("Vertex model name (default text-bison@001): ").strip() or "text-bison@001"
llm = VertexAI(model_name=model_name)
print("LLM wrapper created for model:", model_name)

Vertex model name (default text-bison@001): gemini-2.5-pro
LLM wrapper created for model: gemini-2.5-pro


In [ ]:
# ===============================
# 5️⃣ Global "database" dictionary
# ===============================
donation_entries = {}

In [ ]:
import uuid
from datetime import datetime, timedelta
import re

In [ ]:
# ===============================
# 6️⃣ Define donation intake tool
# ===============================
try:
    from langchain_core.tools import tool as tool_decorator
except Exception:
    from langchain.tools import tool as tool_decorator

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class DonationDetails(BaseModel):
    item_name: str = Field(..., description="Name of the donated food item")
    quantity: int = Field(..., description="Quantity donated")
    unit: str = Field(..., description="Unit of measurement such as kg, liters, pcs")
    expiry_days: int = Field(..., description="Days remaining before expiry")

In [ ]:
parser = PydanticOutputParser(pydantic_object=DonationDetails)

prompt = PromptTemplate(
    template=(
        "Extract structured donation details from the following message:\n"
        "{text}\n\n"
        "{format_instructions}"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
@tool_decorator("donation_intake", return_direct=True)
def donation_intake(text: str) -> str:
    """Extracts and stores donation details using Vertex AI structured output."""
    global donation_entries

    # Build extraction chain
    chain = prompt | llm | parser

    try:
        details = chain.invoke({"text": text})
    except Exception as e:
        return f"Could not parse donation: {str(e)}"

    # Compute expiry date
    expiry_date = datetime.now() + timedelta(days=details.expiry_days)
    expiry_date_str = expiry_date.strftime("%Y-%m-%d")
    days_left = details.expiry_days

    # Urgency and status calculation (new logic)
    if days_left <= 0:
        urgency_score, status = 0, "expired"
    elif days_left == 1:
        urgency_score, status = 5, "available"
    elif 2 <= days_left <= 3:
        urgency_score, status = 4, "available"
    elif 4 <= days_left <= 5:
        urgency_score, status = 3, "available"
    elif 6 <= days_left <= 7:
        urgency_score, status = 2, "available"
    else:
        urgency_score, status = 1, "available"

    # Assign unique entry ID
    entry_id = str(uuid.uuid4())

    # Store in global dict
    donation_entries[entry_id] = {
        "entry_id": entry_id,
        "item_name": details.item_name,
        "quantity": details.quantity,
        "unit": details.unit,
        "expiry_date": expiry_date_str,
        "days_left": days_left,
        "urgency_score": urgency_score,
        "status": status,
    }

    # Response
    return (
        f"Donation registered! ID: {entry_id}, "
        f"Item: {details.item_name}, Quantity: {details.quantity} {details.unit}, "
        f"Expiry in {days_left} days (Urgency: {urgency_score}/5)."
    )

print("Donation Intake tool ready!")

Donation Intake tool ready!


In [ ]:
donation_entries = {}

In [ ]:
# Install dependencies
%pip install -q streamlit pyngrok

In [ ]:
%%writefile donation_agent_app.py
import streamlit as st
import uuid
from datetime import datetime, timedelta

# ========== LangChain & VertexAI setup ==========
try:
    from langchain_google_vertexai import VertexAI
except Exception:
    from langchain_google_vertexai.llms import VertexAI

from google.cloud import aiplatform
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.tools import tool as tool_decorator
from langchain.agents import initialize_agent, AgentType


# ========== Vertex AI Initialization ==========
PROJECT_ID = "food-donation-agent"
REGION = "us-central1"

aiplatform.init(project=PROJECT_ID, location=REGION)

# ========== Vertex AI Model ==========
llm = VertexAI(model_name="gemini-2.5-pro")

# ========== Global store ==========
donation_entries = {}


# ========== Data Model ==========
class DonationDetails(BaseModel):
    item_name: str = Field(..., description="Name of the donated food item")
    quantity: int = Field(..., description="Quantity donated")
    unit: str = Field(..., description="Unit of measurement such as kg, liters, pcs")
    expiry_days: int = Field(..., description="Days remaining before expiry")


# ========== Parser + Prompt ==========
parser = PydanticOutputParser(pydantic_object=DonationDetails)
prompt = PromptTemplate(
    template=(
        "Extract structured donation details from the following message:\n"
        "{text}\n\n"
        "{format_instructions}"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


# ========== Tool ==========
@tool_decorator("donation_intake", return_direct=True)
def donation_intake(text: str) -> str:
    """Extracts and stores donation details using Vertex AI structured output."""
    global donation_entries
    chain = prompt | llm | parser

    try:
        details = chain.invoke({"text": text})
    except Exception as e:
        return f"Could not parse donation: {str(e)}"

    expiry_date = datetime.now() + timedelta(days=details.expiry_days)
    expiry_date_str = expiry_date.strftime("%Y-%m-%d")
    days_left = details.expiry_days

    if days_left <= 0:
        urgency_score, status = 0, "expired"
    elif days_left == 1:
        urgency_score, status = 5, "available"
    elif 2 <= days_left <= 3:
        urgency_score, status = 4, "available"
    elif 4 <= days_left <= 5:
        urgency_score, status = 3, "available"
    elif 6 <= days_left <= 7:
        urgency_score, status = 2, "available"
    else:
        urgency_score, status = 1, "available"

    entry_id = str(uuid.uuid4())
    donation_entries[entry_id] = {
        "entry_id": entry_id,
        "item_name": details.item_name,
        "quantity": details.quantity,
        "unit": details.unit,
        "expiry_date": expiry_date_str,
        "days_left": days_left,
        "urgency_score": urgency_score,
        "status": status,
    }

    return (
        f"Donation registered! ID: {entry_id}, "
        f"Item: {details.item_name}, Quantity: {details.quantity} {details.unit}, "
        f"Expiry in {days_left} days (Urgency: {urgency_score}/5)."
    )


# ========== Agent ==========
agent = initialize_agent(
    tools=[donation_intake],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


# ========== Streamlit UI ==========
def send_message():
    user_input = st.session_state.user_input
    if user_input:
        st.session_state.messages.append({"role": "user", "text": user_input})
        try:
            response = agent.run(user_input)
        except Exception as e:
            response = f"⚠️ Agent error: {e}"
        st.session_state.messages.append({"role": "agent", "text": response})
        st.session_state.user_input = ""


def run_chat_app():
    st.title("🍽️ Food Donation Chat Agent")

    if "messages" not in st.session_state:
        st.session_state.messages = []

    for msg in st.session_state.messages:
        speaker = "You" if msg["role"] == "user" else "Agent"
        st.markdown(f"**{speaker}:** {msg['text']}")

    st.text_input(
        "Type your donation message here:",
        key="user_input",
        on_change=send_message
    )


run_chat_app()

Overwriting donation_agent_app.py


In [ ]:
!streamlit run donation_agent_app.py --server.port 8501 &>/dev/null &

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken YOUR_AUTH_TOKEN

In [ ]:
from pyngrok import ngrok

# Start ngrok tunnel to Streamlit port 8501
public_url = ngrok.connect(8501)
print("Your Streamlit public URL:", public_url)


Your Streamlit public URL: NgrokTunnel: "https://brimless-constance-toilsome.ngrok-free.dev" -> "http://localhost:8501"
